## 檢查GPU

In [ ]:
! nvidia-smi

# Yolo v4

## 下載 Darknet(AlexeyAB)

In [ ]:
! git clone https://github.com/AlexeyAB/darknet.git

## 參數說明
- GPU=1 使用 GPU CUDA
- CUDNN=1 Use cuDNN v5-v7
- CUDNN_HALF=1 Use Tensor Cores (For Titan V / Tesla V100 / DGX-2 and later) Detect 3x , Train 2x
- OPENCV=1 Use OpenCV 3.x/2.4.x - Cemera

In [ ]:
! head darknet/Makefile
! sed -i "s/GPU=0/GPU=1/g" darknet/Makefile
! sed -i "s/CUDNN=0/CUDNN=1/g" darknet/Makefile
! sed -i "s/OPENCV=0/OPENCV=1/g" darknet/Makefile
! head darknet/Makefile

## 編譯 Yolo v4

In [ ]:
! cd darknet ; make

## 使用 Yolo v4

In [ ]:
! cd darknet ; ./darknet 

# 訓練我的口罩偵測權重

## 使用 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp "/content/drive/MyDrive/mask_dataset.zip" /content/

In [ ]:
! unzip /content/mask_dataset.zip > data_unzip.log

In [ ]:
! rm -rf /content/cfg
! mkdir /content/cfg

In [ ]:
import os
datasets = ['/content/mask_dataset/'+ f for f in os.listdir('/content/mask_dataset/') if not f.endswith('.txt')]
len(datasets) * 0.8

## 80% 訓練

In [ ]:
with open('/content/cfg/train.txt', 'w') as f:
    f.write('\n'.join(datasets[0:1600]))

## 20% 測試

In [ ]:
with open('/content/cfg/test.txt', 'w') as f:
    f.write('\n'.join(datasets[1600:]))

In [ ]:
# create obj
import os
labels = ['best', 'bad1', 'bad2', 'worst']
LOCAL_CFG_DIR_PATH = "/content/cfg"
GDRIVE_WEIGHTS_DIR_PATH = "/content/drive/MyDrive/yolov4-tiny-mask-weights"

with open(os.path.join(LOCAL_CFG_DIR_PATH, "mask.names"), 'w') as f:
  f.write("\n".join(labels))

with open(os.path.join(LOCAL_CFG_DIR_PATH, "mask.data"), 'w') as f:
  f.write("classes=%d\n" % (len(labels)))
  f.write("train=%s/train.txt\n" % (LOCAL_CFG_DIR_PATH))
  f.write("valid=%s/test.txt\n" % (LOCAL_CFG_DIR_PATH))
  f.write("names=%s/mask.names\n" % (LOCAL_CFG_DIR_PATH))
  f.write("backup=%s\n" % (GDRIVE_WEIGHTS_DIR_PATH))

In [ ]:
! head /content/cfg/mask.data
! head /content/cfg/mask.names

In [ ]:
! cp /content/darknet/cfg/yolov4-tiny-custom.cfg /content/cfg/

## 第 20 行 max_batches 改成 8000(4*2000)

In [ ]:
! sed -n -e 20p /content/cfg/yolov4-tiny-custom.cfg
! sed -i '20s/500200/8000/' /content/cfg/yolov4-tiny-custom.cfg
! sed -n -e 20p /content/cfg/yolov4-tiny-custom.cfg

## 第 22 行 steps 改成 max_batches 的 80% and 90%, steps=6400,7200

In [ ]:
! sed -n -e 22p /content/cfg/yolov4-tiny-custom.cfg
! sed -i '22s/400000,450000/6400,7200/' /content/cfg/yolov4-tiny-custom.cfg
! sed -n -e 22p /content/cfg/yolov4-tiny-custom.cfg

## 第 8, 9 行 改成 width=416, height=416

In [ ]:
! sed -n -e 8p -e 9p /content/cfg/yolov4-tiny-custom.cfg
! sed -i '8s/608/416/' /content/cfg/yolov4-tiny-custom.cfg
! sed -i '9s/608/416/' /content/cfg/yolov4-tiny-custom.cfg
! sed -n -e 8p -e 9p /content/cfg/yolov4-tiny-custom.cfg

## 第 220, 269 行 classes 改成 4

In [ ]:
! sed -n -e 220p -e 269p /content/cfg/yolov4-tiny-custom.cfg
! sed -i '220s/80/4/' /content/cfg/yolov4-tiny-custom.cfg
! sed -i '269s/80/4/' /content/cfg/yolov4-tiny-custom.cfg
! sed -n -e 220p -e 269p  /content/cfg/yolov4-tiny-custom.cfg

## 第 212, 263 行 filters 改成 (classes + 5)x3 = (4 + 5) * 3 = 27

In [ ]:
! sed -n -e 212p -e 263p /content/cfg/yolov4-tiny-custom.cfg
! sed -i '212s/255/27/' /content/cfg/yolov4-tiny-custom.cfg
! sed -i '263s/255/27/' /content/cfg/yolov4-tiny-custom.cfg
! sed -n -e 212p -e 263p /content/cfg/yolov4-tiny-custom.cfg

## 下載預先訓練的權重
yolov4-tiny.conv.29

In [ ]:
! wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

## 第一次開始訓練

In [ ]:
! darknet/darknet detector train /content/cfg/mask.data /content/cfg/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show

## 上次斷掉的地方繼續訓練

In [ ]:
! darknet/darknet detector train /content/cfg/mask.data /content/cfg/yolov4-tiny-custom.cfg /content/drive/MyDrive/yolov4-tiny-mask-weights/yolov4-tiny-custom_last.weights -dont_show